In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, f1_score, roc_auc_score, roc_curve, confusion_matrix, ConfusionMatrixDisplay, auc, accuracy_score, classification_report
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import xgboost as xgb

In [5]:
# Load the clustered DRA008156 data
abundance_data = pd.read_feather('C:/Users/odesa/OneDrive - University of Toronto/CRC/LatestDataJan/DRA008156/clustered_complete_DRA008156.feather')
meta_data = pd.read_feather('C:/Users/odesa/OneDrive - University of Toronto/CRC/LatestDataJan/DRA008156/DRA008156_metadata_complete.feather')